In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras import optimizers
import keras
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import MinMaxScaler 

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import tensorflow as tf
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix

from timeit import default_timer as timer

path = '../datasets/'

In [9]:
def metrics(true,predicted):
    #CM = confusion_matrix(predicted, true)
    p = precision_score(true,predicted,average='macro',zero_division=1)
    r = recall_score(true,predicted,average='macro')
    a = accuracy_score(true,predicted)
    f1 = f1_score(true,predicted,average='macro')
    
    
    print("F1 =",f1)
    print("Accuracy =",a)
    print("Precision =",p)
    print("Recall =",r)


def Aamirg(X,y,n_input_nodes,n_hidden_nodes,n_output_nodes,f):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42,shuffle = True)

    model = models.Sequential()
    model.add(layers.Dense(n_input_nodes, activation=f, input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(n_hidden_nodes, activation=f))
    model.add(layers.Dense(n_output_nodes,activation='softmax'))
    #callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    batch = 1
    opt = optimizers.rmsprop_v2.RMSprop(learning_rate=0.0001)
    model.compile(optimizer=opt,
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    start = timer()
    model.fit(X_train,
    y_train,
    epochs=int( len(X_train) / batch),
    batch_size= batch,
    validation_data=(X_test,y_test),
    #callbacks=[callback],
    verbose = False)
    end = timer()
    print("TIME",end-start)
    pred = np.argmax(model.predict(X_test), axis=-1)
    tt = np.argmax(y_test, axis=-1)
#     cm = confusion_matrix(tt, pred)
#     sns.heatmap(cm, annot=True)
    metrics(tt,pred)   
    
    return model

In [10]:
#POKEMON DATASET
dataset = "Pokemon.csv"
data = pd.read_csv(path + dataset)


n_input_nodes = 10 #CANTIDAD DE ATRIBUTOS
n_output_nodes = 3 #CANTIDAD DE CLASES


data = data.set_index("#")

d = {}
for i in range(len(data["Type 2"].unique())):
  d[data["Type 2"].unique()[i]] = i + 1
data = data.drop(columns = ["Name"])
data = data.fillna(0)
data["Type 2"] = data["Type 2"].replace(d)


data_balanced = data[(data["Type 1"] == "Water") | (data["Type 1"] == "Normal") | (data["Type 1"] == "Grass")]
m = {'Water':0,'Normal':1,'Grass':2}
data_balanced = data_balanced.replace({'Type 1':m})




y = data_balanced["Type 1"].to_numpy()
data_balanced.drop(['Type 1'],inplace=True,axis=1)

means = np.mean(data_balanced,axis=0)
stds = np.std(data_balanced,axis=0)
raw = (data_balanced-means)/stds

X = data_balanced.to_numpy()
X = np.asarray(X).astype('float32')
t = to_categorical(y, num_classes=n_output_nodes) 

min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)


n_hidden_nodes = 25
print("\nn_hidden_nodes =",n_hidden_nodes,"SIGMOID")
model = Aamirg(X,t,n_input_nodes,n_hidden_nodes,n_output_nodes,"sigmoid")

n_hidden_nodes = 50
print("\nn_hidden_nodes =",n_hidden_nodes,"SIGMOID")
model = Aamirg(X,t,n_input_nodes,n_hidden_nodes,n_output_nodes,"sigmoid")

n_hidden_nodes = 100
print("\nn_hidden_nodes =",n_hidden_nodes,"SIGMOID")
model = Aamirg(X,t,n_input_nodes,n_hidden_nodes,n_output_nodes,"sigmoid")

n_hidden_nodes = 25
print("\nn_hidden_nodes =",n_hidden_nodes,"RELU")
model = Aamirg(X,t,n_input_nodes,n_hidden_nodes,n_output_nodes,"relu")

n_hidden_nodes = 50
print("\nn_hidden_nodes =",n_hidden_nodes,"RELU")
model = Aamirg(X,t,n_input_nodes,n_hidden_nodes,n_output_nodes,"relu")

n_hidden_nodes = 100
print("\nn_hidden_nodes =",n_hidden_nodes,"RELU")
model = Aamirg(X,t,n_input_nodes,n_hidden_nodes,n_output_nodes,"relu")


n_hidden_nodes = 25 SIGMOID
TIME 18.05598759999998
F1 = 0.18974358974358974
Accuracy = 0.3978494623655914
Precision = 0.7992831541218638
Recall = 0.3333333333333333

n_hidden_nodes = 50 SIGMOID
TIME 19.111613399999897
F1 = 0.25866666666666666
Accuracy = 0.43010752688172044
Precision = 0.7363636363636363
Recall = 0.36241956241956247

n_hidden_nodes = 100 SIGMOID
TIME 19.623854800000117
F1 = 0.20973298880275626
Accuracy = 0.40860215053763443
Precision = 0.8007246376811595
Recall = 0.3428571428571428

n_hidden_nodes = 25 RELU
TIME 19.178062100000034
F1 = 0.45603402445507707
Accuracy = 0.5698924731182796
Precision = 0.7254901960784315
Recall = 0.4961818961818962

n_hidden_nodes = 50 RELU
TIME 19.358289000000013
F1 = 0.4933595211372989
Accuracy = 0.5483870967741935
Precision = 0.5030030030030029
Recall = 0.4997854997854998

n_hidden_nodes = 100 RELU
TIME 19.123272000000043
F1 = 0.5033068783068783
Accuracy = 0.5483870967741935
Precision = 0.5084533038021409
Recall = 0.5056199056199057
